Now we will continue on the [Conversation AI](https://conversationai.github.io/) dataset seen in [week 4 homework and lab](https://github.com/MIDS-scaling-up/v2/tree/master/week04). 
We shall use a version of pytorch BERT for classifying comments found at [https://github.com/huggingface/pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT).  

The original implementation of BERT is optimised for TPU. Google released some amazing performance improvements on TPU over GPU, for example, see [here](https://medium.com/@ranko.mosic/googles-bert-nlp-5b2bb1236d78) - *BERT relies on massive compute for pre-training ( 4 days on 4 to 16 Cloud TPUs; pre-training on 8 GPUs would take 40–70 days).*. In response, Nvidia released [apex](https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/), which gave mixed precision training. Weights are stored in float32 format, but calculations, like forward and backward propagation happen in float16 - this allows these calculations to be made with a [4X speed up](https://github.com/huggingface/pytorch-pretrained-BERT/issues/149).  

We shall apply BERT to the problem for classifiying toxicity, using apex from Nvidia. We shall compare the impact of hardware by running the model on a V100 and P100 and comparing the speed and accuracy in both cases.   

This script relies heavily on an existing [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967). 
  
*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

In [175]:
import sys, os
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

Change the PATH variable to whereever your `week06/hw` directory is located.  
**For the final run we would like you to have a train_size of at least 1 Million rows, and a valid size of at least 500K rows. When you first run the script, feel free to work with a reduced train and valid size for speed.** 

In [205]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 1
PATH = '/root/v2/week06/hw' # /root/v2/week06/hw"
DATA_DIR = os.path.join(PATH, "data")
WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
train_size= 1000000 
valid_size= 500000

In [12]:
! bash download.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 61 not upgraded.
--2020-07-19 20:47:45--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.204.240, 172.217.20.144, 172.217.169.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.204.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  43.0MB/s    in 9.8s    

2020-07-19 20:47:55 (39.8 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2020-07-19 20:47:55--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)...

This should be the files you downloaded earlier when you ran `download.sh`

In [178]:
os.listdir(DATA_DIR)

['download.sh',
 'bert_pytorch.bin',
 'cased_L-12_H-768_A-12',
 '.ipynb_checkpoints',
 'uncased_L-12_H-768_A-12',
 'test.csv',
 'train.csv']

We shall install pytorch BERT implementation.   
If you would like to experiment with or view any code (purely optional, and not graded :) ), you can copy the files from the repo https://github.com/huggingface/pytorch-pretrained-BERT  

In [206]:
%%capture
#from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
#from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
#from pytorch_pretrained_bert.modeling import BertModel
#from pytorch_pretrained_bert import BertConfig

from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification

We shall now load the model. When you run this, comment out the `capture` command to understand the archecture.

In [207]:
%%capture
# Translate model from tensorflow to pytorch
#BERT_MODEL_PATH = os.path.join(DATA_DIR, 'uncased_L-12_H-768_A-12')
#convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
#                            os.path.join(BERT_MODEL_PATH, 'bert_model.ckpt'),
#                            os.path.join(BERT_MODEL_PATH, 'bert_config.json'), 
#                            os.path.join(WORK_DIR, 'pytorch_model.bin'))

#shutil.copyfile(os.path.join(BERT_MODEL_PATH, 'bert_config.json'), \
#                os.path.join(WORK_DIR, 'bert_config.json'))

# This is the Bert configuration file
#bert_config = BertConfig(os.path.join(WORK_DIR, 'bert_config.json'))
#listed out default configs
bert_config = BertConfig(vocab_size=30522, hidden_size=768, num_hidden_layers=12, 
                         num_attention_heads=12, intermediate_size=3072, hidden_act='gelu', 
                         hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1, 
                         max_position_embeddings=512, type_vocab_size=2, initializer_range=0.02, 
                         layer_norm_eps=1e-12, pad_token_id=0, gradient_checkpointing=False)

Bert needs a special formatting of sentences, so we have a sentence start and end token, as well as separators.   
Thanks to this [script](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming) for a fast convertor of the sentences.

In [208]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

Now we load the BERT tokenizer and convert the sentences.

In [209]:
%%time
#tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_all = pd.read_csv(os.path.join(DATA_DIR, "train.csv")).sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['comment_text'] = train_all['comment_text'].astype(str) 

sequences = convert_lines(train_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500000 records


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



33724
CPU times: user 31min 29s, sys: 5.46 s, total: 31min 34s
Wall time: 31min 31s


Let us look at how the tokenising works in BERT, see below how it recongizes misspellings - words the model never saw. 

In [247]:
train_all[["comment_text", 'target']].head()

,comment_text,target
458232,It's difficult for many old people to keep up ...,0.0
272766,She recognized that her tiny-handed husband is...,0.0
339129,"HPHY76,\nGood for you for thinking out loud, w...",0.0
773565,And I bet that in the day you expected your Je...,1.0
476233,Kennedy will add a much needed and scientifica...,0.0


Lets tokenize some text (I intentionally mispelled some words to check berts subword information handling)

In [248]:
text = 'Hi, I am learning new things in w251 about deep learning the cloud and teh edge.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i am learning new things in w ##25 ##1 about deep learning the cloud and te ##h edge .'

Added start and end token and convert to ids. This is how it is fed into BERT.

In [249]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2572 4083 2047 2477 1999 1059 17788 2487 2055 2784 4083 1996 6112 1998 8915 2232 3341 1012 102'

When BERT converts this sentence to a torch tensor below is shape of the stored tensors.  
We have 12 input tensors, while the sentence tokens has length 23; where are can you see the 23 tokens in the tensors ?... **Feel free to post in slack or discuss in class**

In [253]:
# put input on gpu and make prediction
# Let's load our model
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_attentions=True).cuda()

# Now just use this line in transformers to extract the loss from the output tuple:
#bert_output = bert(input_ids).cuda()

#bert = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
#bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids]).cuda())
#loss = bert_output[0]

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 15.75 GiB total capacity; 14.68 GiB already allocated; 17.56 MiB free; 14.76 GiB reserved in total by PyTorch)

When the code above actually runs, there is only 1 tensor, not 12, but I am not sure where I am going wrong. I have been using the transformer documentation (instead of pytorch_pretrained_bert) for troubleshooting, but am still running into the CUDA out of memory RuntimeError.

As it is a binary problem, we change our target to [0,1], instead of float.   
We also split the dataset into a training and validation set, 

In [254]:
train_all['target']=(train_all['target']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['target']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['target']].values[train_size:]

In [255]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

**From here on in we would like you to run BERT.**   
**Please do rely on the script available -  [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967) - for at least the first few steps up to training and prediction.**


**1)**   
**Load the training set to a training dataset. For this you need to load the X sequences and y objects to torch tensors**   
**You can use `torch.utils.data.TensorDataset` to input these into a train_dataset.**

In [256]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

**2)**  
**Set your learning rate and batch size; and optionally random seeds if you want reproducable results**   
**Load your pretrained BERT using `BertForSequenceClassification`**   
**Initialise the gradients and place the model on cuda, set up your optimiser and decay parameters**
**Initialise the model with `apex` (we imprted this as `amp`) for mixed precision training**

In [258]:
#parameters
lr=2e-5
batch_size = 32
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
num_warmup_steps = 100

model = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = torch.optim.AdamW(optimizer_grouped_parameters,
                     lr=lr)

# To reproduce BertAdam specific behavior set correct_bias=False
#optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, 
#                                            num_training_steps=num_train_optimization_steps)  # PyTorch scheduler

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model = model.train()

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 15.75 GiB total capacity; 14.68 GiB already allocated; 17.56 MiB free; 14.76 GiB reserved in total by PyTorch)

**3)**  
**Start training your model by iterating through batches in a single epoch of the data**

In [259]:
##
tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
#    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
#    scheduler.step()
    optimizer.zero_grad()
    
    for i,(x_batch, y_batch) in tk0:
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


RuntimeError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 15.75 GiB total capacity; 14.68 GiB already allocated; 17.56 MiB free; 14.76 GiB reserved in total by PyTorch)

**4)**  
**Store your trained model to disk, you will need it if you choose section 8C.**

In [260]:
torch.save(model.state_dict(), output_model_file)

**5)**   
**Now make a prediction for your validation set.**  

In [261]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()

model.load_state_dict(torch.load(output_model_file))
model.to(device)

for param in model.parameters():
    param.requires_grad=False
model.eval()

valid_preds = np.zeros((len(X_val)))

valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 15.75 GiB total capacity; 14.68 GiB already allocated; 17.56 MiB free; 14.76 GiB reserved in total by PyTorch)

**6)**  
**In the yuval's kernel he get a metric based on the metric for the jigsaw competition - it is quite complicated. Instead, we would like you to measure the `AUC`, similar to how you did in homework 04. You can compare the results to HW04**  
*A tip, if your score is lower than homework 04 something is wrong....*

In [262]:
valid_preds
# was curious to see the output, which I see is not populating correctly due to not showing 12 tensors I think?

array([0., 0., 0., ..., 0., 0., 0.])

In [263]:
y_preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
print('AUC score : {:.5f}'.format(roc_auc_score(y_val, y_preds)))

ValueError: Found input variables with inconsistent numbers of samples: [500000, 5000]

When the above code doesnt error out (when I was using a smaller input size for testing, I was only getting an AUC of 0.50000, which is less than hw4 - believe it has to do with how I'm pulling in the bert model.

**7)**  
**Can you show/print the validation sentences predicted with the highest and lowest toxicity ?**

In [225]:
size=5
highest=valid_preds.argsort()[:-size:-1]
highest=highest+train_size
lowest=valid_preds.argsort()[:size]
lowest=lowest+train_size
print ('Highest Toxicity:\n')
for i in highest:
    print(i,train_all.iloc[i].comment_text,'\n')

print ('\n')
    
print ('Lowest Toxicity:\n')
for i in lowest:
    print(i,train_all.iloc[i].comment_text,'\n')

Highest Toxicity:

1004999 "Liberals" won't call you racist, but Indigenous peoples and those farther left might.  That's not the same as Liberals though... 

1001669 So as long as experienced teachers with masters degrees and national board certifications fall above the poverty line - you think they're being paid a fair wage? Get a clue buddy. 

1001662 That's easy. All Incumbants , all Republican. 

1001663 Incorrect. Her counter-suit was for $1. She said if the jury awarded more she would donate that to charity. 



Lowest Toxicity:

1000000 it has to do with the "last mile" 

trains are better at moving stuff long distances- more efficient than trucks 

BUT it's that last mile  - the distance from the train depot to delivery where trucks outperform trains

 it is more efficient to leave everything on the truck than move it train to truck - lower prices make people like you happy

so now trucks are hauling more stuff due to the last mile problem 

more trucks on the highway  means m

**8)**  
**Pick only one of the below items and complete it. The last two will take a good amount of time (and partial success on them is fine), so proceed with caution on your choice of items :)** 
  
  
**A. Can you train on two epochs ?**

**B. Can you change the learning rate and improve validation score ?**
   
**C. Make a prediction on the test data set with your downloaded model and submit to Kaggle to see where you score on public LB - check out [Abhishek's](https://www.kaggle.com/abhishek) script - https://www.kaggle.com/abhishek/pytorch-bert-inference . Note, you will need to fork Abhisheks kernel, swap out the weights to your downloaded weights and commit the kernel. When finalised and you get the output, there is a button to submit to the competition**  
  
**D. Get BERT running on the tx2 for a sample of the data.** 
  
**E. Finally, and very challenging -- the `BertAdam` optimiser proved to be suboptimal for this task. There is a better optimiser for this dataset in this script [here](https://www.kaggle.com/cristinasierra/pretext-lstm-tuning-v3). Check out the `custom_loss` function. Can you implement it ? It means getting under the hood of the `BertForSequenceClassification` at the source repo and implementing a modified version locally .  `https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/modeling.py`**

### Question 8
Attempting to train on 2 epochs, but keep getting the CUDA out of memory error.